In [1]:
import pandas as pd
import requests
from pprint import pprint as pp
from api_keys import g_key

popFile2019_csv = "Resources/co_est2019_alldata.csv"
population_df = pd.read_csv(popFile2019_csv,encoding='latin-1')
population_df = population_df[["STNAME","CTYNAME","POPESTIMATE2019"]]
population_df = population_df.rename(columns={'STNAME': 'State', 'CTYNAME': 'County'})
population_df.head()

,State,County,POPESTIMATE2019
0,Alabama,Alabama,4903185
1,Alabama,Autauga County,55869
2,Alabama,Baldwin County,223234
3,Alabama,Barbour County,24686
4,Alabama,Bibb County,22394


In [2]:
medIncome_csv = "Resources/2018 ACS 5 Year Detailed Median Household Income.csv"
medIncome_df = pd.read_csv(medIncome_csv) 
medIncome_df.head()

,GEO_ID,NAME,B19013_001E,B19013_001M
0,id,Geographic Area Name,Estimate!!Median household income in the past ...,Margin of Error!!Median household income in th...
1,0500000US30019,"Daniels County, Montana",54750,2497
2,0500000US72073,"Jayuya Municipio, Puerto Rico",15424,2978
3,0500000US39173,"Wood County, Ohio",59789,1513
4,0500000US24023,"Garrett County, Maryland",49619,1948


In [3]:
medIncSt = []
medIncCty = []

new = medIncome_df["NAME"].str.split(", ", n = 1, expand = True) 
medIncSt = new[1]
medIncCty = new[0]
medIncome_df["State"] = medIncSt
medIncome_df["County"] = medIncCty
medIncome_df = medIncome_df[["GEO_ID","State","County","B19013_001E", "B19013_001M" ]]
medIncome_df = medIncome_df.rename(columns={'B19013_001E': 'Est. Med HH Income (2018 Dollars)' , 'B19013_001M': 'Margin of Error'})
medIncome_df.head()

,GEO_ID,State,County,Est. Med HH Income (2018 Dollars),Margin of Error
0,id,None,Geographic Area Name,Estimate!!Median household income in the past ...,Margin of Error!!Median household income in th...
1,0500000US30019,Montana,Daniels County,54750,2497
2,0500000US72073,Puerto Rico,Jayuya Municipio,15424,2978
3,0500000US39173,Ohio,Wood County,59789,1513
4,0500000US24023,Maryland,Garrett County,49619,1948


In [4]:
censusData_df  = pd.merge(left=population_df, right=medIncome_df, on=['State','County'], how='inner')
censusData_df = censusData_df[['GEO_ID', 'State', 'County', 'POPESTIMATE2019', 'Est. Med HH Income (2018 Dollars)', 'Margin of Error']]
censusData_df.head()

,GEO_ID,State,County,POPESTIMATE2019,Est. Med HH Income (2018 Dollars),Margin of Error
0,0500000US01001,Alabama,Autauga County,55869,58786,2972
1,0500000US01003,Alabama,Baldwin County,223234,55962,1204
2,0500000US01005,Alabama,Barbour County,24686,34186,2552
3,0500000US01007,Alabama,Bibb County,22394,45340,5618
4,0500000US01009,Alabama,Blount County,57826,48695,2703


In [5]:
censusData_df['Lat'] = ""
censusData_df['Lng'] = ""
lat = []
lng = []

for i, row in censusData_df.iterrows():
    rowCty = row["County"]
    rowSte = row["State"]
    target_cty = f"{rowCty}, {rowSte}"
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
        'address={0}&key={1}').format(target_cty, g_key)
    geo_data = requests.get(target_url).json()
    lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
    lng.append(geo_data["results"][0]["geometry"]["location"]["lng"])

censusData_df['Lat'] = lat
censusData_df['Lng'] = lng
    
#pp(geo_data)

In [6]:
censusData_df

,GEO_ID,State,County,POPESTIMATE2019,Est. Med HH Income (2018 Dollars),Margin of Error,Lat,Lng
0,0500000US01001,Alabama,Autauga County,55869,58786,2972,32.579182,-86.499655
1,0500000US01003,Alabama,Baldwin County,223234,55962,1204,30.601074,-87.776333
2,0500000US01005,Alabama,Barbour County,24686,34186,2552,31.817290,-85.354965
3,0500000US01007,Alabama,Bibb County,22394,45340,5618,32.956280,-87.142289
4,0500000US01009,Alabama,Blount County,57826,48695,2703,34.014515,-86.499655
...,...,...,...,...,...,...,...,...
3137,0500000US56037,Wyoming,Sweetwater County,42343,73008,3133,41.851940,-109.188005
3138,0500000US56039,Wyoming,Teton County,23464,83831,5822,43.853632,-110.631449
3139,0500000US56041,Wyoming,Uinta County,20226,58235,5671,41.210740,-110.616892
3140,0500000US56043,Wyoming,Washakie County,7805,53426,3654,43.834783,-107.703763


In [7]:
censusData_df.to_csv('censusData_df_v1.csv', index=False)